In [312]:
import warnings
warnings.filterwarnings('ignore')

In [313]:
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle, randint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [314]:
df = pd.read_csv('Breast_cancer_data.csv')
print(df.head())
print(df.shape)

   mean_radius  mean_texture  mean_perimeter  mean_area  mean_smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   diagnosis  
0          0  
1          0  
2          0  
3          0  
4          0  
(569, 6)


In [315]:
features=['mean_radius','mean_texture','mean_perimeter','mean_area','mean_smoothness']
target=['diagnosis']

In [316]:
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [317]:
X = df[features]
X.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness
0,17.99,10.38,122.80,1001.0,0.11840
1,20.57,17.77,132.90,1326.0,0.08474
2,19.69,21.25,130.00,1203.0,0.10960
3,11.42,20.38,77.58,386.1,0.14250
4,20.29,14.34,135.10,1297.0,0.10030


In [318]:
y = df[target]
y.head()

,diagnosis
0,0
1,0
2,0
3,0
4,0


In [319]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3)


In [320]:
X_test.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness
263,15.61,19.38,100.00,758.6,0.07840
295,13.77,13.27,88.06,582.7,0.09198
19,13.54,14.36,87.46,566.3,0.09779
535,20.55,20.86,137.80,1308.0,0.10460
55,11.52,18.75,73.34,409.0,0.09524


In [321]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [322]:
lgb_reg_params = {
    'learning_rate':    hp.uniform('learning_rate',0.1,5),
    'max_depth':        hp.choice('max_depth',        np.arange(1, 50, 1, dtype=int)),
    'colsample_bytree': hp.uniform('colsample_bytree',0.01,1),
    'num_leaves':       hp.choice('num_leaves',       np.arange(1, 200, 1, dtype=int)),
    'n_estimators':     hp.choice('n_estimators',       np.arange(1, 1000, 1, dtype=int))
}

In [323]:
def f(params):
  lgbm = lgb.LGBMRegressor(n_jobs=-1,early_stopping_rounds=None,**params)
  score = cross_val_score(lgbm, X_train, y_train, cv=2).mean()
  return score

In [324]:
trials = Trials()

In [325]:
startTime = time.time()
result = fmin(
    fn=f,                           # objective function
    space=lgb_reg_params,   # parameter space
    algo=tpe.suggest,               # surrogate algorithm
    max_evals=5000,                  # no. of evaluations
    trials=trials                   # trials object that keeps track of the sample results (optional)
)
print(result)
model=lgb.LGBMClassifier()
model.set_params(**result)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
best_acc= accuracy_score(y_pred,y_test)
execTime=time.time() - startTime
print("Best Acc: ",best_acc)
print("Execution Time (sec):",execTime)

100%|██████████| 5000/5000 [16:34<00:00,  5.03trial/s, best loss: -7.2424498669259016e+78] 
{'colsample_bytree': 0.41634783822198057, 'learning_rate': 4.994401806668531, 'max_depth': 2, 'n_estimators': 997, 'num_leaves': 1}


LightGBMError: Check failed: (num_leaves) > (1) at D:\a\1\s\python-package\compile\src\io\config_auto.cpp, line 334 .
